In [40]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

In [1]:

# load data
articles = pd.read_csv('articles.csv')
customers = pd.read_csv('customers.csv')
transactions_train = pd.read_csv('transactions_train.csv')
transactions_train['customer_id'] = transactions_train['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
customers['customer_id'] = customers['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
# handle missing values
articles.fillna(value=0, inplace=True)
customers.fillna(value=0, inplace=True)
transactions_train.fillna(value=0, inplace=True)






In [2]:
transactions_train = transactions_train.drop_duplicates(subset=['customer_id','article_id']).reset_index(drop = True)

In [3]:
customer_list = transactions_train.customer_id.value_counts()[:50].index.tolist()

In [4]:
customers_1 = customers.loc[customers['customer_id'].isin(customer_list)]
transactions_train_t = transactions_train.loc[transactions_train['customer_id'].isin(customer_list)]

In [5]:
transactions_train_test = transactions_train_t.loc[transactions_train_t['t_dat']>'2020-04-01'] 

In [6]:
transactions_train_train = transactions_train_t.loc[transactions_train_t['t_dat']<='2020-04-01'] 

In [7]:
transactions_train_test

,t_dat,customer_id,article_id,price,sales_channel_id
20587642,2020-04-02,1135991499650384534,857570001,0.033881,2
20587643,2020-04-02,1135991499650384534,869423001,0.084729,2
20588614,2020-04-02,-738104704126742956,730683041,0.033881,2
20589542,2020-04-02,-3864861336650713034,786304006,0.025407,2
20598202,2020-04-02,-4557035136527070563,857812004,0.042034,2
...,...,...,...,...,...
27304077,2020-09-22,2768447994533782618,889661001,0.066186,1
27304078,2020-09-22,2768447994533782618,865799005,0.033085,1
27304079,2020-09-22,2768447994533782618,918525001,0.049627,1
27304974,2020-09-22,2636587246054202986,752814020,0.033881,2


In [8]:
transactions_train_train

,t_dat,customer_id,article_id,price,sales_channel_id
666,2018-09-20,1135991499650384534,668766002,0.042356,2
667,2018-09-20,1135991499650384534,652946001,0.050831,2
668,2018-09-20,1135991499650384534,691275008,0.067780,2
2958,2018-09-20,-3864861336650713034,501820043,0.016932,2
2959,2018-09-20,-3864861336650713034,501820045,0.016932,2
...,...,...,...,...,...
20583796,2020-04-01,-7961016853555269047,671646003,0.028153,2
20583797,2020-04-01,-7961016853555269047,640202001,0.013169,2
20583798,2020-04-01,-7961016853555269047,739612001,0.011576,2
20583799,2020-04-01,-7961016853555269047,831069001,0.009932,2


In [9]:
articles_t = articles.loc[articles['article_id'].isin(list(transactions_train_t['article_id']))]

In [11]:
merge_df_train = pd.DataFrame()
for customer in customers_1['customer_id']:
    new_df = pd.DataFrame()
    new_df['Buy'] = []
    print(customer)
    new_df = new_df.append([customers_1.loc[customers_1['customer_id']==customer]]*len(articles_t)).reset_index(drop = True)
    new_df['article_id'] = list(articles_t['article_id'])
    tran_temp = transactions_train_train.loc[transactions_train_train['customer_id']==customer]
    for article in list(tran_temp['article_id']):
        new_df.loc[(new_df['customer_id']==customer) & (new_df['article_id']==article),['Buy']] = 1
    merge_df_train = merge_df_train.append(new_df).reset_index(drop = True)


1135991499650384534
-738104704126742956
8572405339616664040
-3864861336650713034
-7724193906943787184
8775840678685643411
1137836553762819430
6025578233842836240
7952062577868193503
3267368325910590416
-8030009826304683278
-5697361604447432642
742206143892321423
-4557035136527070563
628088449402505029
-8684636470525030339
-1277389631032871377
8170076110479575893
-1897419710364364417
-1719797630016463240
-6737281167552191327
3319755920444815383
2697331706436144899
-1229325526706752666
-5976676472876429720
4181321428134864254
9070597775879917454
-6943172096873938826
-2385229251180014276
8683271572215123548
3407358910964148684
-1219804509442908655
7398229172292340849
-8298629768350782975
5854009424779598107
-4781325834093528838
1362310019195253974
-1021296598342089594
-2110333707337082135
5345619263166536794
-8742378727249606979
-20833052798049461
1575517627660529403
-2021215128221699234
-7961016853555269047
-2954818649882845510
2768447994533782618
6876265344195936652
5876564334685098354


In [15]:
merge_df_test = pd.DataFrame()
for customer in customers_1['customer_id']:
    new_df = pd.DataFrame()
    new_df['Buy'] = []
    print(customer)
    new_df = new_df.append([customers_1.loc[customers_1['customer_id']==customer]]*len(articles_t)).reset_index(drop = True)
    new_df['article_id'] = list(articles_t['article_id'])
    tran_temp = transactions_train_test.loc[transactions_train_test['customer_id']==customer]
    for article in list(tran_temp['article_id']):
        new_df.loc[(new_df['customer_id']==customer) & (new_df['article_id']==article),['Buy']] = 1
    merge_df_test = merge_df_train.append(new_df).reset_index(drop = True)


1135991499650384534
-738104704126742956
8572405339616664040
-3864861336650713034
-7724193906943787184
8775840678685643411
1137836553762819430
6025578233842836240
7952062577868193503
3267368325910590416
-8030009826304683278
-5697361604447432642
742206143892321423
-4557035136527070563
628088449402505029
-8684636470525030339
-1277389631032871377
8170076110479575893
-1897419710364364417
-1719797630016463240
-6737281167552191327
3319755920444815383
2697331706436144899
-1229325526706752666
-5976676472876429720
4181321428134864254
9070597775879917454
-6943172096873938826
-2385229251180014276
8683271572215123548
3407358910964148684
-1219804509442908655
7398229172292340849
-8298629768350782975
5854009424779598107
-4781325834093528838
1362310019195253974
-1021296598342089594
-2110333707337082135
5345619263166536794
-8742378727249606979
-20833052798049461
1575517627660529403
-2021215128221699234
-7961016853555269047
-2954818649882845510
2768447994533782618
6876265344195936652
5876564334685098354


In [13]:
merge_df_train.fillna(value=0, inplace=True)

In [14]:
merge_df_train.Buy.value_counts()

0.0    1118817
1.0      27383
Name: Buy, dtype: int64

In [16]:
merge_df_test.fillna(value=0, inplace=True)

In [17]:
merge_df_test.Buy.value_counts()

0.0    1141516
1.0      27608
Name: Buy, dtype: int64

In [18]:
merge_df_test= pd.merge(merge_df_test, articles, on='article_id', how='left')
merge_df_train= pd.merge(merge_df_test, articles, on='article_id', how='left')

In [27]:
from keras.preprocessing.text import Tokenizer
import re
from keras.preprocessing.sequence import pad_sequences
text=merge_df_train.drop(columns=['article_id','customer_id','Buy']).apply(lambda x:'||'.join(x.astype(str)),axis=1)
# define text cleaning process
def cleantext(text):
    ## Remove puncuation
    text=re.sub('[!\"#$%&\'()*+,-./:;<=>?@[\\\\]^_`{|}~]',' ',text).lower().split()

    return ' '.join(text)
corpus_text=text.map(lambda x:cleantext(x))
merge_df_train['product_info']=corpus_text
# parameters
## the vocabulary size of texts
vocab_size = 10000
## number of dimensions in the embeddings
embedding_dim = 64
## the maximum length of texts
#150 #200
max_length = 250
## truncating type
trunc_type = 'post'
## padding type
padding_type = 'post'
## Out Of Vocab token - this will replace any unknown words with a token of our choosing
oov_tok = '<OOV>'
# convert X_train to a sequence
# more details
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(corpus_text)
# if x_train is loaded from csv, then it should be x_train['0'], the same is true for texts_to_sequences

word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# tokenize and pad X_train
corpus_text_sequences = tokenizer.texts_to_sequences(corpus_text)
corpus_text_padded = pad_sequences(corpus_text_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [34]:
text_test=merge_df_test.drop(columns=['article_id','customer_id','Buy']).apply(lambda x:'||'.join(x.astype(str)),axis=1)
# define text cleaning process

corpus_text_test=text_test.map(lambda x:cleantext(x))
merge_df_test['product_info']=corpus_text_test
# parameters
## the vocabulary size of texts
vocab_size = 10000
## number of dimensions in the embeddings
embedding_dim = 64
## the maximum length of texts
#150 #200
max_length = 250
## truncating type
trunc_type = 'post'
## padding type
padding_type = 'post'
## Out Of Vocab token - this will replace any unknown words with a token of our choosing
oov_tok = '<OOV>'
# convert X_train to a sequence
# more details
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(corpus_text_test)
# if x_train is loaded from csv, then it should be x_train['0'], the same is true for texts_to_sequences

word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# tokenize and pad X_train
corpus_text_sequences_test = tokenizer.texts_to_sequences(corpus_text_test)
corpus_text_padded_test = pad_sequences(corpus_text_sequences_test, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [57]:
num_users = len(merge_df_train['customer_id'].unique())
num_items = len(merge_df_train['article_id'].unique())
embedding_size = 32

user_input = Input(shape=(1,))
user_embedding = Embedding(num_users, embedding_size)(user_input)
user_embedding = Flatten()(user_embedding)

item_input = Input(shape=(250,))
item_embedding = Embedding(num_items, embedding_size)(item_input)
item_embedding = Flatten()(item_embedding)

concat = Concatenate()([user_embedding, item_embedding])
hidden_1 = Dense(64, activation='relu')(concat)
hidden_2 = Dense(32, activation='relu')(hidden_1)
output = Dense(1, activation='sigmoid')(hidden_2)

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [58]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 250)]        0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 1, 32)        1600        input_11[0][0]                   
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 250, 32)      733568      input_12[0][0]                   
____________________________________________________________________________________________

In [29]:
corpus_text_padded.shape

(1169124, 250)

In [48]:
from tensorflow.keras.callbacks import ModelCheckpoint

best_weights = ModelCheckpoint('best_model.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto')

In [59]:
history = model.fit([merge_df_train['customer_id'].values,corpus_text_padded], merge_df_train['Buy'].values, epochs=5, validation_data=([merge_df_test['customer_id'].values,corpus_text_padded], merge_df_test['Buy'].values),callbacks=[best_weights])

Epoch 1/5
36536/36536 [==============================] - 472s 13ms/step - loss: 0.1113 - accuracy: 0.9764 - val_loss: 0.1101 - val_accuracy: 0.9764

Epoch 00001: val_accuracy improved from -inf to 0.97639, saving model to best_model.h5
Epoch 2/5
36536/36536 [==============================] - 485s 13ms/step - loss: 0.1094 - accuracy: 0.9764 - val_loss: 0.1077 - val_accuracy: 0.9764

Epoch 00002: val_accuracy did not improve from 0.97639
Epoch 3/5
36536/36536 [==============================] - 474s 13ms/step - loss: 0.1088 - accuracy: 0.9764 - val_loss: 0.1073 - val_accuracy: 0.9764

Epoch 00003: val_accuracy did not improve from 0.97639
Epoch 4/5
36536/36536 [==============================] - 479s 13ms/step - loss: 0.1085 - accuracy: 0.9764 - val_loss: 0.1069 - val_accuracy: 0.9764

Epoch 00004: val_accuracy did not improve from 0.97639
Epoch 5/5
36536/36536 [==============================] - 493s 13ms/step - loss: 0.1082 - accuracy: 0.9764 - val_loss: 0.1067 - val_accuracy: 0.9764

Epo

In [60]:
results = pd.read_csv('sample_submission.csv')

In [62]:
results['customer_id'] = results['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

In [67]:
results = results.loc[results['customer_id'].isin(customer_list)]

In [35]:
predicted_probabilities = model.predict(corpus_text_padded_test)


In [69]:
merge_df_test['prediction'] = predicted_probabilities

In [70]:
merge_df_test

,Buy,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,article_id,product_code,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,product_info,prediction
0,0.0,1.135991e+18,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,108775015,108775,...,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,1.0||1.0||active||regularly||51.0||8db52856d17...,0.966492
1,1.0,1.135991e+18,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,108775044,108775,...,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,1.0||1.0||active||regularly||51.0||8db52856d17...,0.966311
2,0.0,1.135991e+18,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,110065001,110065,...,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",1.0||1.0||active||regularly||51.0||8db52856d17...,0.923191
3,0.0,1.135991e+18,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,110065011,110065,...,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",1.0||1.0||active||regularly||51.0||8db52856d17...,0.927968
4,0.0,1.135991e+18,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,111565001,111565,...,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo...",1.0||1.0||active||regularly||51.0||8db52856d17...,0.917935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169119,0.0,2.636587e+18,1.0,1.0,ACTIVE,Regularly,70.0,3455b39b24a47ae0262c91c5728ab9ddcfccc43628291e...,946795001,946795,...,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1012,Skirts,Calf-length skirt in flock-print mesh with a g...,1.0||1.0||active||regularly||70.0||3455b39b24a...,0.865768
1169120,0.0,2.636587e+18,1.0,1.0,ACTIVE,Regularly,70.0,3455b39b24a47ae0262c91c5728ab9ddcfccc43628291e...,947509001,947509,...,Ladies Accessories,1,Ladieswear,65,Womens Big accessories,1019,Accessories,Narrow belt in leather with a metal buckle. Th...,1.0||1.0||active||regularly||70.0||3455b39b24a...,0.889808
1169121,0.0,2.636587e+18,1.0,1.0,ACTIVE,Regularly,70.0,3455b39b24a47ae0262c91c5728ab9ddcfccc43628291e...,947934001,947934,...,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights in soft lace with an elasticated waist.,1.0||1.0||active||regularly||70.0||3455b39b24a...,0.966800
1169122,0.0,2.636587e+18,1.0,1.0,ACTIVE,Regularly,70.0,3455b39b24a47ae0262c91c5728ab9ddcfccc43628291e...,952267001,952267,...,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Fine-knit tights with an elasticated waist tha...,1.0||1.0||active||regularly||70.0||3455b39b24a...,0.901272


In [78]:

for customer in customers_1['customer_id']:
    customer_dfs = merge_df_test.loc[merge_df_test['customer_id'] == customer]
    customer_not_bought = customer_dfs.loc[customer_dfs['Buy']==0]
    customer_not_bought = customer_not_bought.sort_values('prediction')
    customer_not_bought_top12 = customer_not_bought.head(12)
    customer_not_bought_top12_list = list(customer_not_bought_top12.article_id)
    results.loc[results['customer_id'] == customer,['top12']] = str(customer_not_bought_top12_list).replace(',', ' ').strip('[').strip(']')

C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set 

C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set 

C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Suhan Gautam\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set 

In [84]:
results = results.reset_index(drop = True)

In [86]:
results

,customer_id,prediction,top12
0,1135991499650384534,0706016001 0706016002 0372860001 0610776002 07...,672456007 672456001 186372042 186372045 63...
1,-738104704126742956,0706016001 0706016002 0372860001 0610776002 07...,672456007 672456001 186372042 186372045 81...
2,8572405339616664040,0706016001 0706016002 0372860001 0610776002 07...,672456007 672456001 806527004 583534011 18...
3,-3864861336650713034,0706016001 0706016002 0372860001 0610776002 07...,672456007 672456001 186372042 186372045 80...
4,-7724193906943787184,0706016001 0706016002 0372860001 0610776002 07...,727711002 727711001 721481001 752468001 64...
5,8775840678685643411,0706016001 0706016002 0372860001 0610776002 07...,642631001 642631002 721481001 742038005 72...
6,1137836553762819430,0706016001 0706016002 0372860001 0610776002 07...,672456007 672456001 186372045 186372042 58...
7,6025578233842836240,0706016001 0706016002 0372860001 0610776002 07...,716671004 752468001 642631001 642631002 69...
8,7952062577868193503,0706016001 0706016002 0372860001 0610776002 07...,672456007 672456001 186372042 186372045 18...
9,3267368325910590416,0706016001 0706016002 0372860001 0610776002 07...,642631001 752468001 642631002 716671004 76...
